# Laboratory №6

### Task 1

In [86]:
import csv
import numpy as np
import sympy as sym

from matplotlib import pyplot as plt

file = "input.csv"

N = 3
EPS = 1e-4

class Func:

    def __init__(self, x, y, B, C, D, replace = False):
        self.size = np.size(x)

        self.x = x
        self.y = y
        self.B = B.flatten()
        self.C = C.flatten()
        self.D = D.flatten()

        self.replace = replace

    def getIndex(self, x0):

        if x0 < self.x[0]:
            return 0

        for i in range(1, self.size - 1):
            if self.x[i - 1] < x0 < self.x[i + 1]: return i

        return self.size - 2

    def calculate(self, x0):

        ind = self.getIndex(x0)

        ans = self.y[ind] + self.B[ind] * (x0 - self.x[ind]) + self.C[ind] * (x0 - self.x[ind]) ** 2 + self.D[ind] * (x0 - self.x[ind]) ** 3

        if self.replace: return sym.exp(ans)
        else: return ans

In [89]:
def readFromFile(file):
    Table = []

    with open(file) as f:
        reader = csv.DictReader(f)

        for row in reader:
            Table.append(np.array(list(map(float, [*row.values()]))))

    return Table

def quadratureFormula(n: int):
    if n % 2 == 0: return 2 / (n + 1) 
    else: return 0

def getLegendrePoly(num: int):

    x = sym.symbols("x")
    func = f"(x**2-1)**{num} / (2 ** {num})"
    func = sym.sympify(func)
    func = func / sym.factorial(num)

    return sym.Derivative(func, (x, num)).doit()

def solveSystem(polyZeros):

    matrix = np.array([polyZeros ** i for i in range(N)], dtype = "float")
    matrix.reshape((N, N))

    BCoefs = np.array([quadratureFormula(i) for i in range(N)], dtype = "float")
    BCoefs.reshape((N, 1))

    return np.linalg.solve(matrix, BCoefs)

def Gauss(a, b, f: Func, ACoefs, polyZeros):

    xValues = (b + a) / 2 + (b - a) * polyZeros / 2

    funcValues = np.array([f.calculate(x) for x in xValues])

    return (b - a) * np.sum(ACoefs * funcValues) / 2

def Simpson(a, b, f: Func):
    
    h = (b - a) / N
    edge = int(N / 2)
    xValues = np.linspace(a, b, 2 * N + 1)

    return h * np.sum([f.calculate(xValues[2 * i]) + 4 * f.calculate(xValues[2 * i + 1]) + f.calculate(xValues[2 * i + 2]) for i in range(edge)]) / 3

poly = getLegendrePoly(N)
polyZeros = np.array(sym.solve(poly.doit(), sym.Symbol('x')))
polyZeros = np.sort(polyZeros) 

# polyZeros - нули полинома Лежандра
# ACoefs - A коэффициенты

ACoefs = solveSystem(polyZeros)

# ans = solveIntegral(-2, 2, func, ACoefs, polyZeros)
# print(ans)

# Table = np.array(readFromFile(file))
# x, y = np.meshgrid(Table[:, 0], Table[:, 0])
# z = Table[:, 2:]

# ax = plt.axes(projection="3d")
# ax.plot_surface(x, y, z, cmap = "viridis")
# plt.show()

def spline(x, y):

    size = np.size(x)

    deltaX = np.diff(x)
    deltaY = np.diff(y)

    # A * C = b

    A = np.zeros((size, size))
    b = np.zeros((size, 1))
    A[0, 0] = 1
    A[-1, -1] = 1

    for i in range(1, size - 1):
        A[i, i - 1] = deltaX[i - 1]
        A[i, i + 1] = deltaX[i]
        A[i,i] = 2*(deltaX[i-1] + deltaX[i])

        b[i,0] = 3*(deltaY[i] / deltaX[i] - deltaY[i - 1] / deltaX[i - 1])

    C = np.linalg.solve(A, b)

    D = np.zeros((size - 1, 1))
    B = np.zeros((size - 1, 1))

    for i in range(0, size - 1):
        D[i] = (C[i + 1] - C[i]) / (3 * deltaX[i])
        B[i] = (deltaY[i] / deltaX[i]) - (deltaX[i] / 3)*(2 * C[i] + C[i + 1])

    return B, C, D

def solutionWithoutReplace(Table, ACoefs, polyZeros):

    integrals = []
    x = Table[:, 0]
    size = np.shape(Table)[1]

    for i in range(2, size):

        z = Table[:, i]
        B, C, D = spline(x, z)

        print(x, z)

        f = Func(x, z, B, C, D)

        integrals.append(Gauss(0, 1 - x[i - 2], f, ACoefs, polyZeros))
    
    integrals = np.array(integrals, dtype = "float")

    y = Table[:, 1]
    B, C, D = spline(y, integrals)

    f = Func(y, integrals, B, C, D)

    return Simpson(0, 1, f)

def solutionWithReplace(Table, ACoefs, polyZeros):

    integrals = []
    x = Table[:, 0]
    size = np.shape(Table)[1]

    for i in range(2, size):

        z = np.log(Table[:, i])
        B, C, D = spline(x, z)

        f = Func(x, z, B, C, D)

        integrals.append(Gauss(0, 1 - x[i - 2], f, ACoefs, polyZeros))
            
    integrals = np.array(integrals, dtype = "float")

    y = Table[:, 1]
    B, C, D = spline(y, integrals)

    f = Func(y, integrals, B, C, D, True)

    return Simpson(0, 1, f)

Table = np.array(readFromFile(file))

for i in range(13):
    for j in range(13):
        Table[i, j + 2] = 2 * Table[i, 0] + 3 * Table[j, 1] + 3

ans = solutionWithoutReplace(Table, ACoefs, polyZeros)
print(f"Integral without replacement: {ans:.6f}")
ans = solutionWithReplace(Table, ACoefs, polyZeros)
print(f"Integral with replacement: {ans:.6f}")

[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [3.  3.2 3.4 3.6 3.8 4.  4.2 4.4 4.6 4.8 5.  5.2 5.4]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [3.3 3.5 3.7 3.9 4.1 4.3 4.5 4.7 4.9 5.1 5.3 5.5 5.7]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [3.6 3.8 4.  4.2 4.4 4.6 4.8 5.  5.2 5.4 5.6 5.8 6. ]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [3.9 4.1 4.3 4.5 4.7 4.9 5.1 5.3 5.5 5.7 5.9 6.1 6.3]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [4.2 4.4 4.6 4.8 5.  5.2 5.4 5.6 5.8 6.  6.2 6.4 6.6]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [4.5 4.7 4.9 5.1 5.3 5.5 5.7 5.9 6.1 6.3 6.5 6.7 6.9]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [4.8 5.  5.2 5.4 5.6 5.8 6.  6.2 6.4 6.6 6.8 7.  7.2]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [5.1 5.3 5.5 5.7 5.9 6.1 6.3 6.5 6.7 6.9 7.1 7.3 7.5]
[0.  0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2] [5.4 5.6 5.8 6.  6.2 6.4 6.6 6.8 7.  7.2 7.4 7.6 7.8]
[0.  0.1 0.2 0.3 0.4 0.5 0.6

In [153]:
def func1(x):
    return x ** 2

def Simpson(a, b, func):
    
    h = (b - a) / N
    edge = int(N / 2)
    xValues = np.linspace(a, b, 2 * N + 1)

    return h * np.sum([func(xValues[2 * i]) + 4 * func(xValues[2 * i + 1]) + func(xValues[2 * i + 2]) for i in range(edge)]) / 3


5.333333333333333


### Task 2

In [165]:
import numpy as np
import prettytable as pt

N = 6

def formatOut(num):
    return f"{num:.5f}"

def leftDiffDer(yValue, step, index):
    if index > 0:
        return formatOut((yValue[index] - yValue[index - 1]) / step)
    else:
        return '*'

def centerDiffDer(yValue, step, index):
    if index > 0 and index < N - 1:
        return formatOut((yValue[index + 1] - yValue[index - 1]) / (2 * step))
    else:
        return '*'

def secondRunge(yValue, step, index):
    if index < 2:
        return '*'
        
    f1 = (yValue[index] - yValue[index - 1]) / (step)
    f2 = (yValue[index] - yValue[index - 2]) / (2 * step)

    return formatOut(2 * f1 - f2)

def aligVars(yValue, xValue, index):
    if index > N - 2:
        return '*'

    d = (1 / yValue[index + 1] - 1 / yValue[index]) / \
        (1 / xValue[index + 1] - 1 / xValue[index])

    return formatOut(d * yValue[index] ** 2 / xValue[index] ** 2)

def secondDiffDer(yValue, step, index):
    if index > 0 and index < N - 1:
        return formatOut((yValue[index - 1] - 2 * yValue[index] + yValue[index + 1]) / step ** 2)
    else:
        return ' *'

xArr = [1, 2, 3, 4, 5, 6]
yArr = [0.571, 0.889, 1.091, 1.231, 1.333, 1.412]
    
step = (xArr[-1] - xArr[0]) / len(xArr)

methods = [leftDiffDer, centerDiffDer, 
           secondRunge, aligVars, 
           secondDiffDer]

In [166]:
table = pt.PrettyTable()
filedNames = ["X", "Y", "1", "2", "3", "4", "5"]

table.add_column(filedNames[0], xArr)
table.add_column(filedNames[1], yArr)

for i in range(len(methods)):
    if i == 3:
        table.add_column(filedNames[i + 1], [methods[i](yArr, xArr, j) for j in range(N)])
    else:    
        table.add_column(filedNames[i + 2], [methods[i](yArr, step, j) for j in range(N)])

print(table)

+---+-------+---------+---------+---------+---------+----------+
| X |   Y   |    1    |    2    |    3    |    3    |    5     |
+---+-------+---------+---------+---------+---------+----------+
| 1 | 0.571 |    *    |    *    |    *    | 0.40850 |     *    |
| 2 | 0.889 | 0.38160 | 0.31200 |    *    | 0.24690 | -0.16704 |
| 3 | 1.091 | 0.24240 | 0.20520 | 0.17280 | 0.16544 | -0.08928 |
| 4 | 1.231 | 0.16800 | 0.14520 | 0.13080 | 0.11774 | -0.05472 |
| 5 | 1.333 | 0.12240 | 0.10860 | 0.09960 | 0.08950 | -0.03312 |
| 6 | 1.412 | 0.09480 |    *    | 0.08100 |    *    |     *    |
+---+-------+---------+---------+---------+---------+----------+
